<a href="https://colab.research.google.com/github/plnu-biomechanics/kin6015/blob/main/notebooks/kin6015_lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://www.pointloma.edu/sites/default/files/styles/basic_page/public/images/PLNU_Biomechanics_Lab_green_yellowSD_HiRes.png" width=400>

## **KIN 6015 Biomechanical Basis of Human Movement**
Instructor: Arnel Aguinaldo, PhD

**Lab 6 Data Processing**

In this lab, CMJ and SJ data was collected with muscle activation patterns for the rectus femoris, hamstring, and gastroc-soleus complex using surface electromyography (EMG). Inverse kinematics (IK) and kinetics via inverse dynamics as well as rectified EMG were estimated using Visual3D. The data were then exported as text (*.txt) files and uploaded to the class repository in the lab's [GitHub](https://github.com/plnu-biomechanics).

To further process the data for this lab, follow the steps in this **Colab notebook**, which contains instructions and sample code on how to wrangle and analyze the data.


### Create your own Colab Notebook

1. Go to **File -> New notebook in Drive** to open a new notebook in your Python environment:<br>
<img src="https://raw.githubusercontent.com/plnu-biomechanics/kin6015/main/notebooks/images/file_notebook.png" width=450>

2. Rename your Colab notebook using this naming format: **lastname_group_lab#.ipynb** (e.g., "aguinaldo_targaryen_lab1.ipynb")
3. Click on the **+ Code** option above to insert a new code cell: <br>
<img src="https://raw.githubusercontent.com/plnu-biomechanics/kin6015/main/notebooks/images/addcode.png" width=280>

4. The data you will parse and analyze for this lab will be copied from the lab's GitHub and temporarily stored in your Colab's runtime directory, which can be accessed by clicking on the folder icon in the left menu:<br>
<img src="https://raw.githubusercontent.com/plnu-biomechanics/kin6015/main/notebooks/images/colab_folder.png" width=400>

5. Copy the following lines of code to import the packages needed for this analysis and to load the data files into your working directory. Be sure to update the `GROUP` variable with your group's name. **Note**: These files are "runtime" access only, meaning they are only temporarily stored in your working directory and show up when your notebook is in session. However, the following code cell allows you to clone the zipped files to the working directory each time it is executed.


In [ ]:
import urllib.request
import zipfile
import os

# --------------------------------------------------
# STUDENT INPUT (edit only this line; case-sensitive)
# --------------------------------------------------
GROUP = "stark"   # e.g., "targaryen", "stark", "lannister", "martell", "greyjoy"

# --------------------------------------------------
# Configuration (do NOT edit below)
# These lines create a directory for this lab in your
# Colab working directory.
# --------------------------------------------------
zip_dir = "kin6015/lab6"
os.makedirs(zip_dir, exist_ok=True)

zip_filename = f"spring2026_lab6_{GROUP}.zip"

url = (
    "https://raw.githubusercontent.com/"
    "plnu-biomechanics/kin6015/main/"
    f"labs/{zip_filename}"
)

zip_path = os.path.join(zip_dir, zip_filename)

# --------------------------------------------------
# Download zip file
# --------------------------------------------------
urllib.request.urlretrieve(url, zip_path)

# --------------------------------------------------
# Extract contents from the zipped file
# --------------------------------------------------
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(zip_dir)

print("Extracted files in lab directory:")
print(os.listdir(zip_dir))


## 🔄 Data Parsing

You can add your own code below to parse the data needed for this lab by either using GenAI (e.g., Gemini, ChatGPT) or simply copying the prepared code below.

### ✋ GenAI prompt:
Parse the data from the text files so that they are two unique data frames, one for the timeseries data and another for the discrete metrics. The timeseries data (101 frames) are in the `Vertical_GRF`, `HAMSTRING`, `RECTUS_FEM` and `GASTROC` columns and the discrete metrics are in the remaining columns in row 6 (index 5). Be sure to organize them by CMJ and SJ conditions, which are denoted in the filename as metadata.

### ✅ Pre-compiled code:


In [ ]:
import pandas as pd
import os
import re # Import the regular expression module

def parse_txt_file(filepath):
  """Reads a text file, extracts variable names and time-series data, identifies the condition,
  and returns a pandas DataFrame.

  Args:
    filepath (str): The full path to the text file.

  Returns:
    pd.DataFrame: A DataFrame containing the extracted data, condition, and filename.
  """
  with open(filepath, 'r') as file:
    lines = file.readlines()

  # Check if file has enough lines for the expected structure
  if len(lines) < 6: # Need at least 6 lines (5 header, 1 data)
      raise ValueError(f"File {os.path.basename(filepath)} has too few lines to parse correctly.")

  # Extract variable names from the 2nd line (index 1), using tab as delimiter
  variable_names = lines[1].strip().split('\t')

  # Read time-series data starting from the 6th line (index 5), using tab as delimiter
  data_lines = [line.strip().split('\t') for line in lines[5:] if line.strip()]

  # Ensure data_lines is not empty before checking lengths
  if not data_lines:
      raise ValueError(f"No data found in file {os.path.basename(filepath)} after header.")

  # Dynamically add 'Frame' if data rows have one more column than variable names
  if len(data_lines[0]) == len(variable_names) + 1:
      variable_names.insert(0, 'Frame')

  # Ensure variable names and data lines match in length AFTER potential adjustment
  if len(variable_names) != len(data_lines[0]):
      raise ValueError(f"Column name count ({len(variable_names)}) does not match data column count ({len(data_lines[0])}) in file {os.path.basename(filepath)}.")

  # Create DataFrame
  df = pd.DataFrame(data_lines, columns=variable_names)

    # Rename 'MAX_R_GRF_MEAN' to 'Vertical GRF' if it exists
  if 'GRFz' in df.columns:
    df = df.rename(columns={'GRFz': 'Vertical_GRF'})

  # Convert numeric columns to appropriate data types
  for col in df.columns:
    try:
      df[col] = pd.to_numeric(df[col])
    except ValueError:
      pass # Keep as string if not numeric

  # Determine 'condition' from filename
  filename = os.path.basename(filepath)
  if 'CMJ' in filename:
    condition = 'CMJ'
  elif 'SJ' in filename:
    condition = 'SJ'
  else:
    condition = 'unknown'

  # Add 'condition' column
  df['condition'] = condition

  # Add 'filename' column
  df['filename'] = filename

  return df

print("Updated function 'parse_txt_file' to handle tab-delimited data, correct line indexing, dynamic 'Frame' column, case-sensitive condition check, import 're' module, rename 'Right Ankle Angles' column, and remove '_MEAN'/'_Mean' suffixes.")


# 1. Get the list of .txt files from the working directory
zip_dir = "kin6015/lab6"
all_files = os.listdir(zip_dir)
txt_files = [f for f in all_files if f.endswith('.txt')]

# 2. Initialize an empty list to store DataFrames
all_dataframes = []

# 3. Iterate through each .txt file and apply the parse_txt_file function
for filename in txt_files:
    filepath = os.path.join(zip_dir, filename)
    try:
        df = parse_txt_file(filepath)
        all_dataframes.append(df)
    except Exception as e:
        print(f"Error processing {filename}: {e}")

# 4. Concatenate all DataFrames into a single combined_intermediate_df
if all_dataframes:
    combined_intermediate_df = pd.concat(all_dataframes, ignore_index=True)
    print("Combined_intermediate_df created successfully.")
else:
    combined_intermediate_df = pd.DataFrame()
    print("No dataframes to concatenate, combined_intermediate_df is empty.")

# Multiply specified columns by 100, if needed
# combined_intermediate_df['HAMSTRING'] = combined_intermediate_df['HAMSTRING'] * 100
# combined_intermediate_df['RECTUS_FEM'] = combined_intermediate_df['RECTUS_FEM'] * 100
# combined_intermediate_df['GASTROC'] = combined_intermediate_df['GASTROC'] * 100

# 5. Create timeseries_df
timeseries_df = combined_intermediate_df[
    [
        'Frame',
        'Vertical_GRF',
        'HAMSTRING',
        'RECTUS_FEM',
        'GASTROC',
        'condition',
        'filename'
        ]
    ]

# 6. Create discretes_df
discretes_df = combined_intermediate_df[combined_intermediate_df['Frame'] == 1][
    [
        'filename',
        'condition',
        'Jump_Height_cm',
        'PEAK_GRFz'
        ]
    ]

# 7. Display the head of timeseries_df
print("\nHead of timeseries_df:")
print(timeseries_df.head())

# 8. Display the head of discretes_df
print("\nHead of discretes_df:")
print(discretes_df.head())

## 📈 Statistical Parametric Mapping (SPM)

Statistical parametric mapping (SPM) uses Random Field Theory to make statistical inferences about normalized sets of biomechanical variables across time (1D). It can be used to compare kinematic, kinetic, or EMG time-series curves using traditional NHST statistical tests such as a t-test, ANOVA, and linear regression. In this course, we will be using the package, `spm1d`, in [Python](https://spm1d.org/index.html) to perform some basic SPM analyses.

Before we perform the SPM analyses, we must prepare the time-series data so that they can be properly structured for SPM. The process of preparing and organizing raw data into usable data is known as **data wrangling**. For SPM, the time-series data must be structured so that each variable for each condition (i.e., conventional, sumo) is its own data file, where the data is organized in a *JxQ* array:

* J = number of trials or subjects
* Q = number of frames or time points (i.e., 100)

The following code wrangles the time-series data in these arrays and stores them into individual data frames for subsequent SPM analysis. This will allow the `spm1d` package to average the data at each frame across all trials in a process known as **ensemble averaging** where the solid thick lines represent the mean and the semi-transparent bands represent the +/- 1 standard deviation:

<img src="https://raw.githubusercontent.com/plnu-biomechanics/kin6015/main/notebooks/images/SPM_plot_sample.png">

This two-line cell installs the `spm1d` library in runtime environment and confirms installation.

In [ ]:
!pip install spm1d
print("spm1d library installed.")

# Data Wrangling

Now the data from the `timeseries_df` data frame must be extracted and formatted so that they can be properly analyzed using SPM. This is a part of data wrangling and is a common and important part of the data parsing process.

### ✋ GenAI prompt:
Generate code that wrangles the time-series data into a SPM friendly format for the time-series data in this lab. Each data array should be in the shape of j x q where j = 5 (conditions) and q = 101 (frames). There would be a total of 8 separate data arrays:

*   verticalgrf_CMJ
*   verticalgrf_SJ
*   hamstring_CMJ
*   hamstring_SJ
*   rectus_CMJ
*   rectus_SJ
*   gastroc_CMJ
*   gastroc_SJ


### ✅ Pre-compiled code:

In [ ]:
import numpy as np

# ================
# DATA WRANGLING
# ================

# This code chunk wrangles time-series data into respective arrays for SPM analysis

# Recreate timeseries_df with .copy() to avoid SettingWithCopyWarning
timeseries_df = combined_intermediate_df[
    [
        'Frame',
        'Vertical_GRF',
        'HAMSTRING',
        'RECTUS_FEM',
        'GASTROC',
        'condition',
        'filename'
        ]
    ].copy()

# Add 'trial_id' column by cleaning the 'filename'
timeseries_df['trial_id'] = timeseries_df['filename'].str.replace('_CMJ', '', regex=False)
timeseries_df['trial_id'] = timeseries_df['trial_id'].str.replace('_SJ', '', regex=False)
timeseries_df['trial_id'] = timeseries_df['trial_id'].str.replace('.txt', '', regex=False)

print("Head of timeseries_df with new 'trial_id' column:")
display(timeseries_df)

print("\nUnique trial IDs:")
print(timeseries_df.head())

print("\nUnique trial IDs:")
print(timeseries_df['trial_id'].unique())

# export CSV file for inspection (comment if not needed)
# timeseries_df.to_csv('timeseries_data.csv', index=False)
# print("timeseries_df exported to 'timeseries_data.csv'")

# Filter for both conditions
CMJ_df = timeseries_df[timeseries_df['condition'] == 'CMJ']
SJ_df = timeseries_df[timeseries_df['condition'] == 'SJ']

# Prepare data for GRF (CMJ)
verticalgrf_CMJ_data = CMJ_df.pivot(index='trial_id', columns='Frame', values='Vertical_GRF').values

# Prepare data for Verticalulsive (SJ)
verticalgrf_SJ_data = SJ_df.pivot(index='trial_id', columns='Frame', values='Vertical_GRF').values

# Prepare data for Hamstring EMG (CMJ)
hamstring_CMJ_data = CMJ_df.pivot(index='trial_id', columns='Frame', values='HAMSTRING').values

# Prepare data for Hamstring EMG (SJ)
hamstring_SJ_data = SJ_df.pivot(index='trial_id', columns='Frame', values='HAMSTRING').values

# Prepare data for Rectus EMG (CMJ)
rectus_CMJ_data = CMJ_df.pivot(index='trial_id', columns='Frame', values='RECTUS_FEM').values

# Prepare data for Rectus EMG (SJ)
rectus_SJ_data = SJ_df.pivot(index='trial_id', columns='Frame', values='RECTUS_FEM').values

# Prepare data for Gastroc EMG (CMJ)
gastroc_CMJ_data = CMJ_df.pivot(index='trial_id', columns='Frame', values='GASTROC').values

# Prepare data for Rectus EMG (SJ)
gastroc_SJ_data = SJ_df.pivot(index='trial_id', columns='Frame', values='GASTROC').values

# --------------------------------------------------------------------------------------------------
# FIND CMJ TAKE-OFF TIME
# --------------------------------------------------------------------------------------------------

toe_off_frames = [] # Initialize an empty list to store the toe-off frame for each CMJ trial

num_trials = len(verticalgrf_CMJ_data) # Number of trials (rows) in the data
total_frames = verticalgrf_CMJ_data.shape[1] # Total number of frames (columns) in the data
# Loop through each trial (row) in the verticalgrf_CMJ_data
for trial_index in range(num_trials):
    # Loop through each frame from 0 up to total_frames - 5
    # The -5 is to ensure there are enough frames remaining to check for a sequence of 5 zeros
    found_toe_off_for_trial = False
    for frame_index in range(total_frames - 5):
        # Check if the value at the current frame is exactly 0.000
        if verticalgrf_CMJ_data[trial_index, frame_index] == 0.000:
            # Check if the next four consecutive frames are also 0.000
            if np.all(verticalgrf_CMJ_data[trial_index, frame_index : frame_index + 5] == 0.000):
                # If a sequence of five consecutive 0.000 values is found, append the 1-based frame number
                toe_off_frames.append(frame_index + 1)
                found_toe_off_for_trial = True
                break # Break out of the inner (frame) loop to move to the next trial
    if not found_toe_off_for_trial:
        toe_off_frames.append(np.nan) # Append NaN if no toe-off found for the trial

# Calculate the average of the identified toe-off frames, ignoring NaN values
average_toe_off_frame = np.nanmean(toe_off_frames)

# Assign this average to the 'take_off' variable, converting to int if not NaN
if not np.isnan(average_toe_off_frame):
    cmj_TO = int(round(average_toe_off_frame))
else:
    cmj_TO = None # Or handle as appropriate if no valid frames were found

# --------------------------------------------------------------------------------------------------
# FIND CMJ TOUCHDOWN TIME (LANDING) - After CMJ Take-off
# --------------------------------------------------------------------------------------------------

touchdown_frames = [] # Initialize an empty list to store the touchdown (landing) frame for each CMJ trial

if cmj_TO is not None: # Ensure cmj_TO is defined before searching for landing
    for trial_index in range(num_trials):
        found_touchdown_for_trial = False
        # Start searching for touchdown AFTER the cmj_TO event
        # cmj_TO is 1-based, so for 0-based array indexing, we start from cmj_TO.
        for frame_index in range(cmj_TO, total_frames):
            # Check if the GRF value is greater than 0.5 (indicating landing)
            if verticalgrf_CMJ_data[trial_index, frame_index] > 0:
                touchdown_frames.append(frame_index + 1) # Append 1-based frame number
                found_touchdown_for_trial = True
                break # Break out of the inner (frame) loop to move to the next trial
        if not found_touchdown_for_trial:
            touchdown_frames.append(np.nan) # Append NaN if no touchdown (landing) found for the trial
else:
    print("cmj_TO was not found, cannot determine CMJ touchdown (landing) frames.")
    touchdown_frames = [np.nan] * num_trials # Populate with NaNs if cmj_TO is missing

# Calculate the average of the identified touchdown frames, ignoring NaN values
average_touchdown_frame = np.nanmean(touchdown_frames)

# Assign this average to the 'cmj_TD' variable, converting to int if not NaN
if not np.isnan(average_touchdown_frame):
    cmj_TD = int(round(average_touchdown_frame))-1
else:
    cmj_TD = None # Or handle as appropriate if no valid frames were found


# --------------------------------------------------------------------------------------------------
# FIND SJ TAKE-OFF TIME
# --------------------------------------------------------------------------------------------------

toe_off_frames = [] # re-initialize an empty list to store the toe-off frame for each SJ trial

num_trials = len(verticalgrf_SJ_data) # Number of trials (rows) in the data
total_frames = verticalgrf_SJ_data.shape[1] # Total number of frames (columns) in the data
# Loop through each trial (row) in the verticalgrf_SJ_data
for trial_index in range(num_trials):
    # Loop through each frame from 0 up to total_frames - 5
    # The -5 is to ensure there are enough frames remaining to check for a sequence of 5 zeros
    found_toe_off_for_trial = False
    for frame_index in range(total_frames - 5):
        # Check if the value at the current frame is exactly 0.000
        if verticalgrf_SJ_data[trial_index, frame_index] == 0.000:
            # Check if the next four consecutive frames are also 0.000
            if np.all(verticalgrf_SJ_data[trial_index, frame_index : frame_index + 5] == 0.000):
                # If a sequence of five consecutive 0.000 values is found, append the 1-based frame number
                toe_off_frames.append(frame_index + 1)
                found_toe_off_for_trial = True
                break # Break out of the inner (frame) loop to move to the next trial
    if not found_toe_off_for_trial:
        toe_off_frames.append(np.nan) # Append NaN if no toe-off found for the trial

# Calculate the average of the identified toe-off frames, ignoring NaN values
average_toe_off_frame = np.nanmean(toe_off_frames)

# Assign this average to the 'take_off' variable, converting to int if not NaN
if not np.isnan(average_toe_off_frame):
    sj_TO = int(round(average_toe_off_frame))
else:
    sj_TO = None # Or handle as appropriate if no valid frames were found

# --------------------------------------------------------------------------------------------------
# FIND SJ TOUCHDOWN TIME (LANDING) - After SJ Take-off
# --------------------------------------------------------------------------------------------------

touchdown_frames = [] # Initialize an empty list to store the touchdown (landing) frame for each SJ trial

if sj_TO is not None: # Ensure sj_TO is defined before searching for landing
    for trial_index in range(num_trials):
        found_touchdown_for_trial = False
        # Start searching for touchdown AFTER the sj_TO event
        # sj_TO is 1-based, so for 0-based array indexing, we start from sj_TO.
        for frame_index in range(sj_TO, total_frames):
            # Check if the GRF value is greater than 0.5 (indicating landing)
            if verticalgrf_SJ_data[trial_index, frame_index] > 0:
                touchdown_frames.append(frame_index + 1) # Append 1-based frame number
                found_touchdown_for_trial = True
                break # Break out of the inner (frame) loop to move to the next trial
        if not found_touchdown_for_trial:
            touchdown_frames.append(np.nan) # Append NaN if no touchdown (landing) found for the trial
else:
    print("sj_TO was not found, cannot determine SJ touchdown (landing) frames.")
    touchdown_frames = [np.nan] * num_trials # Populate with NaNs if sj_TO is missing

# Calculate the average of the identified touchdown frames, ignoring NaN values
average_touchdown_frame = np.nanmean(touchdown_frames)

# Assign this average to the 'sj_TD' variable, converting to int if not NaN
if not np.isnan(average_touchdown_frame):
    sj_TD = int(round(average_touchdown_frame))-1
else:
    sj_TD = None # Or handle as appropriate if no valid frames were found


print(f"Assigned 'cmj_TO' variable: {cmj_TO}")
print(f"Assigned 'cmj_TD' variable: {cmj_TD}")
print(f"Assigned 'sj_TO' variable: {sj_TO}")
print(f"Assigned 'sj_TD' variable: {sj_TD}")
print("Shape of Vertical_GRF (CMJ) data:", verticalgrf_CMJ_data.shape)
print("Shape of Vertical_GRF (SJ) data:", verticalgrf_SJ_data.shape)
print("Shape of Hamstring EMG (CMJ) data:", hamstring_CMJ_data.shape)
print("Shape of Hamstring EMG (SJ) data:", hamstring_SJ_data.shape)
print("Shape of Rectus EMG (CMJ) data:", rectus_CMJ_data.shape)
print("Shape of Rectus EMG (SJ) data:", rectus_SJ_data.shape)
print("Shape of Gastroc EMG (CMJ) data:", gastroc_CMJ_data.shape)
print("Shape of Gastroc EMG (SJ) data:", gastroc_SJ_data.shape)

# SPM Paired t-test

Now let's perform the SPM paired t-tests on the time-series data in this lab so that we can compare them between conditions (CMJ vs. SJ). The specific code below produces the ensemble-averaged plots for both conditions with shaded bands representing suprathreshold regions where the mean differences between condition are statistically significant (SPM{t} p < 0.05).

### ✋ GenAI prompt:

Produce code that will compare the time-series data wrangled earlier between CMJ and SJ conditions using SPM paired t-tests. Be sure to plot the ensemble-average curves for each variable and condition along with the SPM{t} field. Do not output the SPM{t} fields. Rather highlight the suprathreshold regions with shaded bands.

### ✅ Pre-compiled code:

In [ ]:
import spm1d
import numpy as np
import matplotlib.pyplot as pyplot

total_frames = 101
num_ticks = 6  # Number of ticks (0%, 10%, ..., 100%)

# Define a function to create each plot
def plot_spm(ax, Y1, Y2, label_Y1, label_Y2, take_off1, take_off2, land1, land2, x_label, y_label, y_limits, units, color_Y1, color_Y2, show_legend):
    spm1d.plot.plot_mean_sd(Y1, label=label_Y1, linecolor=color_Y1, facecolor=color_Y1, ax=ax, lw=1.5, alpha=0.3, autoset_ylim=False)
    spm1d.plot.plot_mean_sd(Y2, label=label_Y2, linecolor=color_Y2, facecolor=color_Y2, linestyle='--', lw=1.5, alpha=0.4, ax=ax, autoset_ylim=False)
    ax.axhline(y=0, color='k', linestyle=':')

    # --- Start of fix for zero variance error ---
    # Check for zero variance in Y1 and Y2 at each node and add tiny noise if variance is zero
    Y1_copy = Y1.copy() # Create copies to avoid modifying original global arrays directly
    Y2_copy = Y2.copy()
    for i in range(Y1_copy.shape[1]): # Iterate through each node (column)
        if np.var(Y1_copy[:, i]) == 0:
            Y1_copy[:, i] += np.random.rand(Y1_copy.shape[0]) * 1e-9 # Add tiny noise if variance is zero
        if np.var(Y2_copy[:, i]) == 0:
            Y2_copy[:, i] += np.random.rand(Y2_copy.shape[0]) * 1e-9 # Add tiny noise if variance is zero
    # --- End of fix ---

    # Conduct t-test (using ttest_paired for paired comparison):
    alpha      = 0.05
    t          = spm1d.stats.ttest_paired(Y1_copy, Y2_copy) # Pass Y1_copy and Y2_copy separately
    ti         = t.inference(alpha, two_tailed=True, interp=True)

    # Highlight suprathreshold regions
    for cluster in ti.clusters:
        start, end = cluster.endpoints
        ax.axvspan(start, end, ymin=0, ymax=1.0, color='grey', alpha=0.20)

    # Generate plots
    percentages = np.linspace(0, 100, num_ticks)  # 0%, 10%, ..., 100%
    x_ticks = (percentages / 100) * total_frames  # Normalize frame numbers
    ypos = y_limits[1] + y_limits[1]*.02          # event labels
    ax.set_xticklabels(percentages.astype(int), fontsize=8)  # Set labels as integers (0, 10%, ..., 100%)
    # ax.set_xticks(x_ticks)
    ax.vlines(x=take_off2, ymin=y_limits[0]-100, ymax=y_limits[1]+100, color='r', linestyle='--', lw=1)
    ax.text(take_off2-2, ypos, 'TO', fontsize=8, color='r')
    ax.vlines(x=take_off1, ymin=y_limits[0]-100, ymax=y_limits[1]+100, color='k', linestyle='-', lw=1)
    ax.text(take_off1-2, ypos, 'TO', fontsize=8)
    ax.vlines(x=land2, ymin=y_limits[0]-100, ymax=y_limits[1]+100, color='r', linestyle='--', lw=1)
    ax.text(land2-2, ypos, 'TD', fontsize=8, color='r')
    ax.vlines(x=land1, ymin=y_limits[0]-100, ymax=y_limits[1]+100, color='k', linestyle='-', lw=1)
    ax.text(land1-2, ypos, 'TD', fontsize=8)
    ax.set_xlabel(x_label, fontsize=10, fontweight='bold')
    ax.set_ylabel(y_label, fontsize=10, fontweight='bold') if y_label else ax.set_ylabel("")
    ax.set_xlim(0, 100)
    ax.set_ylim(y_limits)

    if show_legend:
      ax.legend(fontsize=9, loc='upper right', frameon=False)

# Combine both SPM plots (no SPM{t} fields)
pyplot.figure( figsize=(10, 9) ) # Adjusted figsize for a 4 plot grid

# =============================================================================
# Vertical GRF
# =============================================================================

# Replace x and y axes and filename label
xlabel = "% Jump Motion"
ylabel="Vertical GRF"
units = "N"

# YA (black)
YA            = verticalgrf_CMJ_data

# YB (red)
YB            = verticalgrf_SJ_data

# Calculate shared y-axis limits
all_data_for_y_limits = np.concatenate((YA.flatten(), YB.flatten()))
all_data_for_y_limits = all_data_for_y_limits[~np.isnan(all_data_for_y_limits)] # remove NaNs

if len(all_data_for_y_limits) > 0:
    min_val = np.min(all_data_for_y_limits)
    max_val = np.max(all_data_for_y_limits)
    if min_val > 0:
        min_val = 0
else: # If all data is NaN after concatenation and filtering
    min_val = -1.0 # Default to some reasonable range if all data is NaN
    max_val = 1.0
y_limits = [min_val, max_val]

# Subplot 1
ax1 = pyplot.subplot(2, 2, 1) # Changed to 1 row, 3 columns, position 1
plot_spm(
    ax1, YA, YB,
    label_Y1="CMJ",
    label_Y2="SJ",
    take_off1 = cmj_TO,
    take_off2 = sj_TO,
    land1 = cmj_TD,
    land2 = sj_TD,
    x_label = xlabel,
    y_label=f'{ylabel} ({units})',
    y_limits=y_limits,
    units=units,
    color_Y1='k',
    color_Y2='r',
    show_legend=False
)


# =============================================================================
# Hamstring EMG
# =============================================================================

# Replace x and y axes and filename label
# xlabel = "% Jump Motion"
ylabel="Hamstring"
units = "%MVC"

# YA (black)
YA            = hamstring_CMJ_data

# YB (red)
YB            = hamstring_SJ_data

# Calculate shared y-axis limits
all_data_for_y_limits = np.concatenate((YA.flatten(), YB.flatten()))
all_data_for_y_limits = all_data_for_y_limits[~np.isnan(all_data_for_y_limits)] # remove NaNs

if len(all_data_for_y_limits) > 0:
    min_val = np.min(all_data_for_y_limits)
    max_val = np.max(all_data_for_y_limits)
    if min_val > 0:
        min_val = 0
else: # If all data is NaN after concatenation and filtering
    min_val = -1.0 # Default to some reasonable range if all data is NaN
    max_val = 1.0

# min_val = 0 # Removed hardcoded limits
# max_val = 1000 # Removed hardcoded limits
y_limits = [min_val, max_val] # Use dynamically calculated limits

# Subplot 2
ax2 = pyplot.subplot(2, 2, 2) # Changed to 1 row, 3 columns, position 2
plot_spm(
    ax2, YA, YB,
    label_Y1="CMJ",
    label_Y2="SJ",
    take_off1 = cmj_TO,
    take_off2 = sj_TO,
    land1 = cmj_TD,
    land2 = sj_TD,
    x_label = xlabel,
    y_label=f'{ylabel} ({units})',
    y_limits=y_limits,
    units=units,
    color_Y1='k',
    color_Y2='r',
    show_legend=True
)

# =============================================================================
# Rectus EMG
# =============================================================================

# Replace x and y axes and filename label
# xlabel = "% Jump Motion"
ylabel="Rectus EMG"
units = "%MVC"

# YA (black)
YA            = rectus_CMJ_data

# YB            = rectus_SJ_data # This line was commented out in previous execution, uncommenting it for correct execution

YB            = rectus_SJ_data

# Calculate shared y-axis limits
all_data_for_y_limits = np.concatenate((YA.flatten(), YB.flatten()))
all_data_for_y_limits = all_data_for_y_limits[~np.isnan(all_data_for_y_limits)] # remove NaNs

if len(all_data_for_y_limits) > 0:
    min_val = np.min(all_data_for_y_limits)
    max_val = np.max(all_data_for_y_limits)
    if min_val > 0:
        min_val = 0
else: # If all data is NaN after concatenation and filtering
    min_val = -1.0 # Default to some reasonable range if all data is NaN
    max_val = 1.0

# min_val = 0 # Removed hardcoded limits
# max_val = 1000 # Removed hardcoded limits
y_limits = [min_val, max_val] # Use dynamically calculated limits

# Subplot 3
ax3 = pyplot.subplot(2, 2, 3) # Changed to 2 rows, 2 columns, position 1
plot_spm(
    ax3, YA, YB,
    label_Y1="CMJ",
    label_Y2="SJ",
    take_off1 = cmj_TO,
    take_off2 = sj_TO,
    land1 = cmj_TD,
    land2 = sj_TD,
    x_label = xlabel,
    y_label=f'{ylabel} ({units})',
    y_limits=y_limits,
    units=units,
    color_Y1='k',
    color_Y2='r',
    show_legend=False
)

# =============================================================================
# Gastroc EMG
# =============================================================================

# Replace x and y axes and filename label
# xlabel = "% Jump Motion"
ylabel="Gastroc EMG"
units = "%MVC"

# YA (black)
YA            = gastroc_CMJ_data

# YB (red)
YB            = gastroc_SJ_data

# Calculate shared y-axis limits
all_data_for_y_limits = np.concatenate((YA.flatten(), YB.flatten()))
all_data_for_y_limits = all_data_for_y_limits[~np.isnan(all_data_for_y_limits)] # remove NaNs

if len(all_data_for_y_limits) > 0:
    min_val = np.min(all_data_for_y_limits)
    max_val = np.max(all_data_for_y_limits)
    if min_val > 0:
        min_val = 0
else: # If all data is NaN after concatenation and filtering
    min_val = -1.0 # Default to some reasonable range if all data is NaN
    max_val = 1.0

# min_val = 0 # Removed hardcoded limits
# max_val = 1000 # Removed hardcoded limits
y_limits = [min_val, max_val] # Use dynamically calculated limits

# Subplot 4
ax4 = pyplot.subplot(2, 2, 4) # Changed to 2 rows, 2 columns, position 1, now ax4
plot_spm(
    ax4, YA, YB,
    label_Y1="CMJ",
    label_Y2="SJ",
    take_off1 = cmj_TO,
    take_off2 = sj_TO,
    land1 = cmj_TD,
    land2 = sj_TD,
    x_label = xlabel,
    y_label=f'{ylabel} ({units})',
    y_limits=y_limits,
    units=units,
    color_Y1='k',
    color_Y2='r',
    show_legend=False
)

### plot SPM results:
pyplot.tight_layout()
pyplot.show()


# Summary Statistics

Now the discrete metrics (e.g., stride length, speed, etc.) need to be compared between conditions using traditional paired t-tests with the summary statistics displayed in an APA-formatted table.

### ✋ GenAI prompt:

Analyze the `discretes_df` to compare CMJ and SJ conditions. This involves preparing the data by extracting a unique subject identifier, calculating summary statistics (mean, standard deviation, count) for the GRF and EMG metrics grouped by condition, and performing paired t-tests for these variables between the two conditions. Finally, format these results into an APA-style table and summarize the key findings, highlighting any statistically significant differences.

### ✅ Pre-compiled code:



In [ ]:
import pandas as pd
from scipy.stats import ttest_rel

# 1. Create a copy of discretes_df
discretes_for_analysis_df = discretes_df.copy()

# 2. Extract a unique subject identifier (the numerical part of the filename) from the 'filename' column
# Assuming filenames are like 'GROUP_CONDITION_SUBJECTID.txt' (e.g., Martell_TRN_04.txt -> 04)
discretes_for_analysis_df['trial_id'] = discretes_for_analysis_df['filename'].str.extract(r'_(\d+)\.txt$')[0]

# Define the desired column order
# This will effectively drop 'filename' and place 'trial_id' after 'condition'
final_columns = ['condition', 'trial_id'] + [col for col in discretes_for_analysis_df.columns if col not in ['condition', 'trial_id', 'filename']]
discretes_for_analysis_df = discretes_for_analysis_df[final_columns]

# Sort the DataFrame by 'condition' to ensure 'CMJ' comes before 'SJ'
# This uses 'CMJ' and 'SJ' as categories with a specific order
discretes_for_analysis_df['condition'] = pd.Categorical(
    discretes_for_analysis_df['condition'],
    categories=['CMJ', 'SJ'],
    ordered=True
)
discretes_for_analysis_df = discretes_for_analysis_df.sort_values(by=['condition', 'trial_id']).reset_index(drop=True)

# 3. Display the head of the discretes_for_analysis_df to verify the new 'trial_id' column and sorting
print("Head of discretes_for_analysis_df with new 'trial_id' column and sorted conditions:")
print(discretes_for_analysis_df.head())

# 4. Define a list of the variables for which summary statistics need to be calculated
summary_variables = [
        'Jump_Height_cm',
        'PEAK_GRFz'
]

# 5. Group the discretes_for_analysis_df DataFrame by the 'condition' column
# 6. For each of the specified variables, calculate the mean, standard deviation, and count for each 'condition' group
summary_statistics = discretes_for_analysis_df.groupby('condition')[summary_variables].agg(
    ['mean', 'std', 'count']
)

# 7. Store these summary statistics in a new DataFrame (already done by the agg function)
# The result is automatically a DataFrame with a multi-level index/columns which is well-organized.

# 8. Display the resulting summary statistics DataFrame
print("Descriptive Statistics grouped by Condition:")
print(summary_statistics)

# 9. Define a list of the variables for which paired t-tests need to be performed
pair_ttest_variables = [
        'Jump_Height_cm',
        'PEAK_GRFz'
]

# 10. Initialize a dictionary to store the t-test results
t_test_results = {}

# 11. For each variable in the defined list:
for var in pair_ttest_variables:
    # a. Filter discretes_for_analysis_df to get data for the 'CMJ' condition.
    CMJ_data = discretes_for_analysis_df[discretes_for_analysis_df['condition'] == 'CMJ'][['trial_id', var]]

    # b. Filter discretes_for_analysis_df to get data for the 'SJ' condition.
    SJ_data = discretes_for_analysis_df[discretes_for_analysis_df['condition'] == 'SJ'][['trial_id', var]]

    # c. Ensure that both filtered DataFrames contain the same 'trial_id's and are sorted by 'trial_id'
    # to guarantee correct pairing.
    merged_data = pd.merge(CMJ_data, SJ_data, on='trial_id', suffixes=('_CMJ', '_SJ'))

    # Drop rows where either CMJ or SJ data is NaN for the current variable
    merged_data = merged_data.dropna(subset=[var + '_CMJ', var + '_SJ'])

    # Extract the values for the current variable from both filtered DataFrames.
    data_CMJ = merged_data[var + '_CMJ']
    data_SJ = merged_data[var + '_SJ']

    # d. Perform a paired t-test using ttest_rel on these two sets of values.
    # Check if there are enough samples to perform the t-test
    if len(data_CMJ) >= 2 and len(data_SJ) >= 2:
        t_statistic, p_value = ttest_rel(data_CMJ, data_SJ)
    else:
        t_statistic, p_value = float('nan'), float('nan') # Set to NaN if not enough samples

    # e. Store the resulting t-statistic and p-value.
    t_test_results[var] = {'t_statistic': t_statistic, 'p_value': p_value}

# 12. Print or display the collected t-test results for each variable.
print("\nPaired t-test results (CMJ vs. SJ):")
for var, results in t_test_results.items():
    print(f"Variable: {var}")
    print(f"  t-statistic: {results['t_statistic']:.3f}")
    print(f"  p-value: {results['p_value']:.3f}")
    print("-----------------------------------------")
# 13. Initialize an empty list to store the data for the APA-style table
apa_table_data = []

# 14. Iterate through each variable in the pair_ttest_variables list
for var in pair_ttest_variables:
    # 3. For each variable, extract the mean and standard deviation for 'CMJ' and 'SJ' conditions
    #    from the summary_statistics DataFrame.
    mean_CMJ = summary_statistics.loc['CMJ', (var, 'mean')]
    std_CMJ = summary_statistics.loc['CMJ', (var, 'std')]
    mean_SJ = summary_statistics.loc['SJ', (var, 'mean')]
    std_SJ = summary_statistics.loc['SJ', (var, 'std')]

    # 4. Extract the t-statistic and p-value for the current variable from the t_test_results dictionary.
    t_stat = t_test_results[var]['t_statistic']
    p_val = t_test_results[var]['p_value']

    # 5. Create a dictionary for the current row with appropriate keys and formatted numerical values.
    row = {
        'Variable': var,
        'M \u00b1 SD (CMJ)': f"{mean_CMJ:.2f} \u00b1 {std_CMJ:.2f}",
         # 'SD (CMJ)': f"{std_CMJ:.2f}",
        'M \u00b1 SD (SJ)': f"{mean_SJ:.2f} \u00b1 {std_SJ:.2f}",
         # 'SD (SJ)': f"{std_SJ:.2f}",
         # 't': f"{t_stat:.3f}",
        'p': f"{p_val:.3f}"
    }

    # Append this row dictionary to the list.
    apa_table_data.append(row)

# 15. Create a pandas DataFrame from the list of rows.
apa_table_df = pd.DataFrame(apa_table_data)

print("APA-Style Summary and Paired t-test Results Table:")
print(apa_table_df.to_string(index=False))

### 👏 Complete the Analysis

After you've completed this lab's analysis, do the following:

1. Copy and paste the above plots to your lab report
2. Export the tables using the code in the following cell. You can download them from your runtime directory onto your own computer.
2. Share your Colab notebook with me
3. Copy the link to your notebook and submit it along with your report for this lab on Canvas

In [ ]:

import os

# Construct the full file path for the CSV file
output_filepath = os.path.join(zip_dir, f"{GROUP}_discretes_for_analysis_lab6.csv")

# Export the DataFrame to CSV
discretes_for_analysis_df.to_csv(output_filepath, index=False)

# Print a confirmation message
print(f"'discretes_for_analysis_df' exported to '{output_filepath}'")

# Construct the full file path for the CSV file
output_filepath = os.path.join(zip_dir, f"{GROUP}_summary_stats_lab6.csv")

# Export the APA formatted summary DataFrame to CSV
apa_table_df.to_csv(output_filepath, index=False)

# Print a confirmation message
print(f"'summary_statistics' exported to '{output_filepath}'")